In [56]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
!pip3 install tensorflow >> /dev/null

In [68]:
import pandas as pd
from dataset_lib import load_dataset, split_dataset
import tensorflow as tf


In [15]:
df = load_dataset()


Putting together /mnt/f/proyectos/bikes/src/models/../../data/parts/chicago-divvy-trips-part-1
Putting together /mnt/f/proyectos/bikes/src/models/../../data/parts/chicago-divvy-trips-part-2
Putting together /mnt/f/proyectos/bikes/src/models/../../data/parts/chicago-divvy-trips-part-3
Putting together /mnt/f/proyectos/bikes/src/models/../../data/parts/chicago-divvy-trips-part-4
Putting together /mnt/f/proyectos/bikes/src/models/../../data/parts/chicago-divvy-trips-part-5
Putting together /mnt/f/proyectos/bikes/src/models/../../data/parts/chicago-divvy-trips-part-6
Putting together /mnt/f/proyectos/bikes/src/models/../../data/parts/chicago-divvy-trips-part-7
Putting together /mnt/f/proyectos/bikes/src/models/../../data/parts/chicago-divvy-trips-part-8
Putting together /mnt/f/proyectos/bikes/src/models/../../data/parts/chicago-divvy-trips-part-9
Putting together /mnt/f/proyectos/bikes/src/models/../../data/parts/chicago-divvy-trips-part-10
Putting together /mnt/f/proyectos/bikes/src/model

In [59]:
train_df, val_df = split_dataset(df)

Training from 2014-01-01 00:00:00 to 2018-12-31 23:00:00
Validating from 2019-01-01 00:00:00 to 2019-12-31 23:00:00


## Baseline forecast

This prediction will be based on the previous day only

In [60]:
train_df

,from_station_id,quantity
start_time,,
2014-01-01 00:00:00,69,1
2014-01-01 00:00:00,94,1
2014-01-01 01:00:00,94,0
2014-01-01 01:00:00,127,3
2014-01-01 01:00:00,240,1
...,...,...
2018-12-31 23:00:00,333,0
2018-12-31 23:00:00,191,0
2018-12-31 23:00:00,216,0
